In [ ]:
Step 5. Plot AS variants and motif enrichment results 

In [ ]:
Add detailed description here 

In [ ]:
library("data.table")
library(ggplot2)
library(RcisTarget)

In [ ]:
motifName <- readLines("/staging/leuven/stg_00002/lcb/zkalender/melanoma_WGS/MM057/alleleseq/motif_analysis/singletons_names.txt")
# 22936
motifName <- motifName[-1]
motifName <- sapply(motifName, strsplit, "\t")
motifName <- setNames(sapply(motifName, function(x) x[2]), sapply(motifName, function(x) x[1]))

readNformat <- function(matFile)
{
  mat <- fread(matFile, sep="\t", header = TRUE)
  mat <- data.frame(mat, check.names=FALSE)
  row.names(mat) <- mat[,1]
  mat <- as.matrix(mat[,-1])
  if(any(is.na(colnames(mat))))  warning("missing motif names")
  return(mat)
}
ASB_MUT <- readNformat("ASB_MUT_sorted.new_cbust_result.tsv")
ASB_REF <- readNformat("ASB_REF_sorted.new_cbust_result.tsv")
dim(ASB_MUT)
dim(ASB_REF)
ASB_delta_comb<-matrix(nrow=0,ncol=22937)

ASB_MUT_losses<-ASB_MUT[grep("LOSES",rownames(ASB_MUT)),]
ASB_MUT_wins<-ASB_MUT[grep("WINS",rownames(ASB_MUT)),]
ASB_REF_losses<-ASB_REF[rownames(ASB_MUT_losses),]
ASB_REF_wins<-ASB_REF[rownames(ASB_MUT_wins),]
ASB_delta <- ASB_MUT_wins - ASB_REF_wins
ASB_delta_comb<-rbind(ASB_delta_comb,ASB_delta)
ASB_delta <- ASB_REF_losses - ASB_MUT_losses
ASB_delta_comb<-rbind(ASB_delta_comb,ASB_delta)
ASB_delta<-ASB_delta_comb
saveRDS(ASB_delta, "ASB_delta.Rds")

CTRL_MUT<-readNformat("control_MUT_sorted.new_cbust_result.tsv")
CTRL_REF<-readNformat("control_REF_sorted.new_cbust_result.tsv")
dim(CTRL_MUT)
dim(CTRL_REF)
CTRL_delta<-CTRL_MUT - CTRL_REF
saveRDS(CTRL_delta, "CTRL_delta.Rds")
fdr01_global <- quantile(abs(CTRL_delta), probs=0.999);fdr01_global
# 8.135
# 3.28
fdr1_global <- quantile(abs(CTRL_delta), probs=0.99);fdr1_global
# 5.18
# 1.129
# delta threshold 3
nMotifsD3 <- cbind(randomAbs= colSums(abs(CTRL_delta) > 3))
nRand <- nrow(CTRL_delta)
nMotifsD3 <- cbind(nMotifsD3,
                   randomUp=colSums(CTRL_delta > 3),
                   randomDw=colSums(CTRL_delta < -3))

nSignif <- nrow(ASB_delta)
nMotifsD3 <- cbind(nMotifsD3,
                   signifAbs=colSums(abs(ASB_delta) > 3),
                   signifUp=colSums(ASB_delta > 3),
                   signifDw=colSums(ASB_delta < -3))

nMotifsD4 <- cbind(randomAbs= colSums(abs(CTRL_delta) > 4))
nRand <- nrow(CTRL_delta)
nMotifsD4 <- cbind(nMotifsD4,
                   randomUp=colSums(CTRL_delta > 4),
                   randomDw=colSums(CTRL_delta < -4))

nSignif <- nrow(ASB_delta)
nMotifsD4 <- cbind(nMotifsD4,
                   signifAbs=colSums(abs(ASB_delta) > 4),
                   signifUp=colSums(ASB_delta > 4),
                   signifDw=colSums(ASB_delta < -4))

# Loop for calculating the Fisher exact test for each motif
nMotifsD3<-nMotifsD3[rowSums(nMotifsD3)>0,]
nMotifsD4<-nMotifsD4[rowSums(nMotifsD4)>0,]

# cbust threshold 3
fisherTestP_cbust3 <- matrix(0, nrow=nrow(nMotifsD3), ncol=6, dimnames=list(rownames(nMotifsD3),c("Abs_gr","Abs_less","Up_gr","Up_less","Dw_gr","Dw_less")))
fisherTestP_cbust4 <- matrix(0, nrow=nrow(nMotifsD4), ncol=6, dimnames=list(rownames(nMotifsD4),c("Abs_gr","Abs_less","Up_gr","Up_less","Dw_gr","Dw_less")))

# ABSOLUTE - cbust3
for (i in rownames(nMotifsD3))
{
  nSignifMod <- nMotifsD3[i,"signifAbs"]
  nRndMod <- nMotifsD3[i,"randomAbs"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter
  fisherTestP_cbust3[i, "Abs_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust3[i, "Abs_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}

# ABSOLUTE - cbust4
for (i in rownames(nMotifsD4))
{
  nSignifMod <- nMotifsD4[i,"signifAbs"]
  nRndMod <- nMotifsD4[i,"randomAbs"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter
  fisherTestP_cbust4[i, "Abs_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust4[i, "Abs_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}

# UP - cbust3
for (i in rownames(nMotifsD3))
{
  nSignifMod <- nMotifsD3[i,"signifUp"]
  nRndMod <- nMotifsD3[i,"randomUp"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter

  fisherTestP_cbust3[i, "Up_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust3[i, "Up_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}

# UP - cbust4
for (i in rownames(nMotifsD4))
{
  nSignifMod <- nMotifsD4[i,"signifUp"]
  nRndMod <- nMotifsD4[i,"randomUp"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter

  fisherTestP_cbust4[i, "Up_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust4[i, "Up_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}


# DOWN - cbust3
for (i in rownames(nMotifsD3))
{
  nSignifMod <- nMotifsD3[i,"signifDw"]
  nRndMod <- nMotifsD3[i,"randomDw"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter

  fisherTestP_cbust3[i, "Dw_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust3[i, "Dw_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}

# DOWN - cbust4
for (i in rownames(nMotifsD4))
{
  nSignifMod <- nMotifsD4[i,"signifDw"]
  nRndMod <- nMotifsD4[i,"randomDw"]
  mat <- matrix(c(nSignifMod, nSignif-nSignifMod,
                  nRndMod, nRand-nRndMod),
                ncol=2, byrow=T) # t() : does not matter

  fisherTestP_cbust4[i, "Dw_less"] <- fisher.test(mat, alternative = "less")$p.value
  fisherTestP_cbust4[i, "Dw_gr"] <- fisher.test(mat, alternative = "greater")$p.value
}

fisherTestP_cbust3 <- fisherTestP_cbust3[order(fisherTestP_cbust3[,"Abs_gr"]),]
saveRDS(fisherTestP_cbust3, file="fisherTestP_cbust3.Rds")

fisherTestP_cbust4 <- fisherTestP_cbust4[order(fisherTestP_cbust4[,"Abs_gr"]),]
saveRDS(fisherTestP_cbust4, file="fisherTestP_cbust4.Rds")


fisherTest_cbust3_Padj <- apply(fisherTestP_cbust3, 2, p.adjust, method="fdr")
fisherTest_cbust4_Padj <- apply(fisherTestP_cbust4, 2, p.adjust, method="fdr")

saveRDS(fisherTest_cbust3_Padj, file="fisherTest_cbust3_Padj.Rds")
saveRDS(fisherTest_cbust4_Padj, file="fisherTest_cbust4_Padj.Rds")

fisherTest_cbust3_Padj <- cbind(fisherTest_cbust3_Padj, nSignif=rowSums(fisherTest_cbust3_Padj < 0.01), nSignifNoAdj=rowSums(fisherTestP_cbust3 < 0.01))
fisherTestPadj_cbust3_signif <- fisherTest_cbust3_Padj[fisherTest_cbust3_Padj[,"nSignifNoAdj"]>0,, drop=FALSE]

fisherTest_cbust4_Padj <- cbind(fisherTest_cbust4_Padj, nSignif=rowSums(fisherTest_cbust4_Padj < 0.01), nSignifNoAdj=rowSums(fisherTestP_cbust4 < 0.01))
fisherTestPadj_cbust4_signif <- fisherTest_cbust4_Padj[fisherTest_cbust4_Padj[,"nSignifNoAdj"]>0,, drop=FALSE]


ASB_delta_mean <- colMeans(ASB_delta)
maxDelta <- max(abs(ASB_delta_mean))

png("fisherTest_new_cbust_3.png", width=900, height=900, res=96, type="cairo")
plot(cbind(deltaMean=ASB_delta_mean[rownames(fisherTest_cbust3_Padj)],
           "-logAdjPval"=-log(fisherTest_cbust3_Padj[,"Up_gr"])), pch=20, cex=1.5,
     xlim=c(-maxDelta,maxDelta), main="Up_gr")
abline(v=0, col="grey", lty=2)
abline(h=-log(0.05), col="grey", lty=3)
abline(h=-log(0.01), col="darkgrey", lty=2)
dev.off()

png("fisherTest_new_cbust_3_Abs_gr.png", width=900, height=900, res=96, type="cairo")
plot(cbind(deltaMean=ASB_delta_mean[rownames(fisherTest_cbust3_Padj)],
           "-logAdjPval"=-log(fisherTest_cbust3_Padj[,"Abs_gr"])), pch=20, cex=1.5,
     xlim=c(-maxDelta,maxDelta), main="Abs_gr")
abline(v=0, col="grey", lty=2)
abline(h=-log(0.05), col="grey", lty=3)
abline(h=-log(0.01), col="darkgrey", lty=2)
dev.off()

png("fisherTest_new_cbust_4.png", width=900, height=900, res=96, type="cairo")
plot(cbind(deltaMean=ASB_delta_mean[rownames(fisherTest_cbust4_Padj)],
           "-logAdjPval"=-log(fisherTest_cbust4_Padj[,"Up_gr"])), pch=20, cex=1.5,
     xlim=c(-maxDelta,maxDelta), main="Up_gr")
abline(v=0, col="grey", lty=2)
abline(h=-log(0.05), col="grey", lty=3)
abline(h=-log(0.01), col="darkgrey", lty=2)
dev.off()

png("fisherTest_new_cbust_4_Abs_gr.png", width=900, height=900, res=96, type="cairo")
plot(cbind(deltaMean=ASB_delta_mean[rownames(fisherTest_cbust4_Padj)],
           "-logAdjPval"=-log(fisherTest_cbust4_Padj[,"Abs_gr"])), pch=20, cex=1.5,
     xlim=c(-maxDelta,maxDelta), main="Abs_gr")
abline(v=0, col="grey", lty=2)
abline(h=-log(0.05), col="grey", lty=3)
abline(h=-log(0.01), col="darkgrey", lty=2)
dev.off()

# Annotation
motifAnnotations<-importAnnotations("/staging/leuven/stg_00002/lcb/icistarget/data/motif2tf/snapshots/motifs-v8-nr.hgnc-m0.001-o0.0.tbl")

deltaMotifStats_cbust3 <- data.frame(fisherTestPadj_cbust3_signif)
annots <- t(sapply(rownames(deltaMotifStats_cbust3), function(x)
  c(directOrtology=paste0(unlist(motifAnnotations[motif==x & (directAnnotation | inferred_Orthology),"TF"]), collapse=" ; "),
    inferred_simil=paste0(unlist(motifAnnotations[motif==x & !(directAnnotation | inferred_Orthology),"TF"]), collapse=" ; "))))
deltaMotifStats_cbust3 <- data.frame(delta=signif(ASB_delta_mean[rownames(deltaMotifStats_cbust3)],3), deltaMotifStats_cbust3, annots)
deltaMotifStats_cbust3$Abs_gr <- signif(deltaMotifStats_cbust3$Abs_gr,3)
deltaMotifStats_cbust3$Abs_less <- signif(deltaMotifStats_cbust3$Abs_less,3)
deltaMotifStats_cbust3$Up_gr <- signif(deltaMotifStats_cbust3$Up_gr,3)
deltaMotifStats_cbust3$Up_less <- signif(deltaMotifStats_cbust3$Up_less,3)
deltaMotifStats_cbust3$Dw_gr <- signif(deltaMotifStats_cbust3$Dw_gr,3)
deltaMotifStats_cbust3$Dw_less <- signif(deltaMotifStats_cbust3$Dw_less,3)
selCols <- c("delta","Abs_gr", "Abs_less", "Up_gr", "Up_less", "Dw_gr", "Dw_less", "directOrtology", "inferred_simil")
table2show <- deltaMotifStats_cbust3[,selCols, drop=FALSE]
write.table(file="motifs_signif_fisher_cbust3.tsv",table2show,sep="\t",quote=F)
save(table2show, file="motifs_signif_fisher_cbust3.RData")
y <- DT::datatable(RcisTarget::addLogo(data.frame(motif=rownames(table2show), table2show)), escape=FALSE)
DT::saveWidget(y, 'motifs_signif_fisher_cbust3.html', selfcontained=F)
rm(table2show)

deltaMotifStats_cbust4 <- data.frame(fisherTestPadj_cbust4_signif)
annots <- t(sapply(rownames(deltaMotifStats_cbust4), function(x)
  c(directOrtology=paste0(unlist(motifAnnotations[motif==x & (directAnnotation | inferred_Orthology),"TF"]), collapse=" ; "),
    inferred_simil=paste0(unlist(motifAnnotations[motif==x & !(directAnnotation | inferred_Orthology),"TF"]), collapse=" ; "))))
deltaMotifStats_cbust4 <- data.frame(delta=signif(ASB_delta_mean[rownames(deltaMotifStats_cbust4)],3), deltaMotifStats_cbust4, annots)
deltaMotifStats_cbust4$Abs_gr <- signif(deltaMotifStats_cbust4$Abs_gr,3)
deltaMotifStats_cbust4$Abs_less <- signif(deltaMotifStats_cbust4$Abs_less,3)
deltaMotifStats_cbust4$Up_gr <- signif(deltaMotifStats_cbust4$Up_gr,3)
deltaMotifStats_cbust4$Up_less <- signif(deltaMotifStats_cbust4$Up_less,3)
deltaMotifStats_cbust4$Dw_gr <- signif(deltaMotifStats_cbust4$Dw_gr,3)
deltaMotifStats_cbust4$Dw_less <- signif(deltaMotifStats_cbust4$Dw_less,3)
selCols <- c("delta","Abs_gr", "Abs_less", "Up_gr", "Up_less", "Dw_gr", "Dw_less", "directOrtology", "inferred_simil")
table2show <- deltaMotifStats_cbust4[,selCols, drop=FALSE]
write.table(file="motifs_signif_fisher_cbust4.tsv",table2show,sep="\t",quote=F)
save(table2show, file="motifs_signif_fisher_cbust4.RData")
y <- DT::datatable(RcisTarget::addLogo(data.frame(motif=rownames(table2show), table2show)), escape=FALSE)
DT::saveWidget(y, 'motifs_signif_fisher_cbust4.html', selfcontained=F)


table2show<-read.delim(file="motifs_signif_fisher_cbust3.tsv",header=T)
ASB_delta<-readRDS("ASB_delta.Rds")

AP1_motifs<-table2show[union(grep("JUN|FOS|BACH",table2show$directOrtology),grep("JUN|FOS|BACH",table2show$inferred_simil)),]
AP1_motifs<-subset(AP1_motifs,Up_gr<=0.05)
SOX_motifs<-table2show[union(grep("SOX",table2show$directOrtology),grep("SOX",table2show$inferred_simil)),]
SOX_motifs<-subset(SOX_motifs,Up_gr<=0.05)
CTCF_motifs<-table2show[union(grep("CTCF",table2show$directOrtology),grep("CTCF",table2show$inferred_simil)),]
CTCF_motifs<-subset(CTCF_motifs,Up_gr<=0.05)
TFAP_motifs<-table2show[union(grep("TFAP",table2show$directOrtology),grep("TFAP",table2show$inferred_simil)),]
TFAP_motifs<-subset(TFAP_motifs,Up_gr<=0.05)
ETS_motifs<-table2show[union(grep("ETV",table2show$directOrtology),grep("ETV",table2show$inferred_simil)),]
ETS_motifs<-subset(ETS_motifs,Up_gr<=0.05)
MYB_motifs<-table2show[union(grep("MYB",table2show$directOrtology),grep("MYB",table2show$inferred_simil)),]
MYB_motifs<-subset(MYB_motifs,Up_gr<=0.05)
FOXP3_motifs<-table2show[union(grep("FOXP3",table2show$directOrtology),grep("FOXP3",table2show$inferred_simil)),]
FOXP3_motifs<-subset(FOXP3_motifs,Up_gr<=0.05)
CREB3_motifs<-table2show[union(grep("CREB3",table2show$directOrtology),grep("CREB3",table2show$inferred_simil)),]
CREB3_motifs<-subset(CREB3_motifs,Up_gr<=0.05)
RUNX_motifs<-table2show[union(grep("RUNX3|RUNX2",table2show$directOrtology),grep("RUNX3|RUNX2",table2show$inferred_simil)),]
RUNX_motifs<-subset(RUNX_motifs,Up_gr<=0.05)
HOX_motifs<-table2show[union(grep("HOX",table2show$directOrtology),grep("HOX",table2show$inferred_simil)),]
HOX_motifs<-subset(HOX_motifs,Up_gr<=0.05)
TEAD_motifs<-table2show[union(grep("TEAD1|TEAD3|TEAD4",table2show$directOrtology),grep("TEAD1|TEAD3|TEAD4",table2show$inferred_simil)),]
TEAD_motifs<-subset(TEAD_motifs,Up_gr<=0.05)
EBOX_motifs<-table2show[union(grep("USF|MITF|ARNT|CLOCK|MYC|NPAS2|MNT|HES5|HES7",table2show$directOrtology),grep("USF|MITF|ARNT|CLOCK|MYC|NPAS2|MNT|HES5|HES7",table2show$inferred_simil)),]
EBOX_motifs<-subset(EBOX_motifs,Up_gr<=0.05)
SP_motifs<-table2show[union(grep("SP2|SP3|SP3|WT1|EGR1|ZNF740|MECOM|ZNF148|ZNF219|CHURC1|OVOL2|IKZF3|MAZ|KLF4",table2show$directOrtology),grep("SP2|SP3|SP3|WT1|EGR1|ZNF740|MECOM|ZNF148|ZNF219|CHURC1|OVOL2|IKZF3|MAZ|KLF4",table2show$inferred_simil)),]
SP_motifs<-subset(SP_motifs,Up_gr<=0.05)

if (!all(is.na(AP1_motifs))){AP1_motifs$ann<-"AP1"}
if (!all(is.na(SOX_motifs))){SOX_motifs$ann<-"SOX"}
if (!all(is.na(CTCF_motifs))){CTCF_motifs$ann<-"CTCF"}
if (!all(is.na(TFAP_motifs))){TFAP_motifs$ann<-"TFAP"}
if (!all(is.na(ETS_motifs))){ETS_motifs$ann<-"ETS"}
if (!all(is.na(MYB_motifs))){MYB_motifs$ann<-"MYB"}
if (!all(is.na(FOXP3_motifs))){FOXP3_motifs$ann<-"FOXP3"}
if (!all(is.na(CREB3_motifs))){CREB3_motifs$ann<-"CREB3"}
if (!all(is.na(RUNX_motifs))){RUNX_motifs$ann<-"RUNX"}
if (!all(is.na(HOX_motifs))){HOX_motifs$ann<-"HOX"}
if (!all(is.na(TEAD_motifs))){TEAD_motifs$ann<-"TEAD"}
if (!all(is.na(EBOX_motifs))){EBOX_motifs$ann<-"EBOX"}
if (!all(is.na(SP_motifs))){SP_motifs$ann<-"SP"}

colors<-c("red2", "darkgreen", "violet", "yellow3", "deepskyblue","plum1", "limegreen", "olivedrab3", "snow4","darkorange","deeppink","coral4","tomato","royalblue3")
names(colors)<-c("AP1", "CTCF","CREB3","EBOX","ETS","FOXP3","HOX","MYB","Other", "RUNX","SOX","SP","TEAD","TFAP")

motif_ann<-rbind(AP1_motifs, SOX_motifs, CTCF_motifs, TFAP_motifs, ETS_motifs,MYB_motifs, FOXP3_motifs, CREB3_motifs, RUNX_motifs,HOX_motifs, TEAD_motifs,EBOX_motifs, SP_motifs)
motif_ann<-motif_ann[,"ann",drop=F]
ASB_delta_sig_motifs<-ASB_delta[,rownames(subset(table2show,Up_gr<=0.05))]
ASB_delta_sig_motifs_binarized<-apply(ASB_delta_sig_motifs,1,function(x){ifelse(x>=3,1,0)})

a<-as.data.frame(rowSums(ASB_delta_sig_motifs_binarized))
colnames(a)<-"rowsum"

table2plot<-merge(table2show,a,by.x="row.names",by.y="row.names",all.x=T)
table2plot<-merge(table2plot, motif_ann, by.x="Row.names",by.y="row.names", all.x=T)

table2plot$rowsum[is.na(table2plot$rowsum)]<- 0.1
table2plot$ann[is.na(table2plot$ann)]<-"Other"

ASB_delta_mean <- colMeans(ASB_delta)
maxDelta <- max(abs(ASB_delta_mean))


png(file="fisherTest_new_cbust_3_ggplot.png",width=1200, height=1200,res=120, type="cairo")
ggplot(table2plot, aes(delta, -log(Up_gr), color=ann, size=rowsum)) +
xlim(-maxDelta,maxDelta) +
geom_point(alpha=0.9) +
scale_colour_manual(values = colors[levels(as.factor(table2plot$ann))] ) + geom_hline(yintercept=-log(0.05),linetype=2,col="grey") +
geom_hline(yintercept=-log(0.01),linetype=2,col="grey") +
geom_vline(xintercept=0,linetype=2,col="grey") + theme_bw() +
theme(legend.title=element_blank(),
    legend.text=element_text(size=12),
    axis.text.x=element_text(size=12),
    axis.text.y=element_text(size=12),
    title=element_text(size=16)) + guides(color = guide_legend(override.aes = list(size=6))) +
    annotate("text", label=paste(paste(dim(ASB_MUT)[1],"test", sep=" "), paste(dim(CTRL_MUT)[1],"control",sep=" "),sep=","), x=-maxDelta+0.05, y=0 )
dev.off()